In [ ]:
import os
import sys

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import seaborn as sns
import pandas as pd

sys.path.append('../')
from src.data_handler import DataHandler
from src.analysis_logic import AnalysisLogic, FitMethods
from src.helper_functions import baseline_als

sns.set_theme()

In [ ]:
data_folder = os.path.join("C:\\", "Data")
figure_folder = os.path.join("C:\\", "5f12dd6ff78b78000186706b", "images", "ch5")

In [ ]:
magnet_handler = DataHandler(
    data_folder=data_folder,
    figure_folder=figure_folder,
    measurement_folder="20210316_MagnetCooldown",
    copy_measurement_folder_structure=False
)

In [ ]:
magnet_handler.data_folder_tree()

In [ ]:
df = magnet_handler.read_pfeiffer_data(os.path.join(magnet_handler.data_folder_path, "pressure-monitoring01.txt"))

In [ ]:
df = df[(df.index.day == 17) & (df.index.hour >= 13) & (df.index.hour <= 16)]

In [ ]:
fig, ax = plt.subplots(nrows=1, figsize=(5, 3))

time_in_hours = (df.index - df.index[0]) / pd.Timedelta(value=1, unit="h")
sns.lineplot(x=time_in_hours, y="CH 1", data=df, ax=ax)

ax.set_yscale("log")
ax.set_xlabel("Time (in hours)")
ax.set_ylabel("Pressure (mbar)")

ax.axvspan(xmin=time_in_hours[100], xmax=time_in_hours[250], color="C3", alpha=0.5, label="Run 1")
ax.axvspan(xmin=time_in_hours[1000], xmax=time_in_hours[1280], color="C3", alpha=0.5, label="Run 2")
ax.axvspan(xmin=time_in_hours[2200], xmax=time_in_hours[2550], color="C3", alpha=0.5, label="Run 3")

fig.tight_layout()
magnet_handler.save_figures(fig, "magnet_quench_pressure", only_pdf=True)

In [ ]:
df2 = magnet_handler.read_pfeiffer_data(os.path.join(magnet_handler.data_folder_path, "pressure-monitoring02.txt"))

In [ ]:
fig, ax = plt.subplots(nrows=1, figsize=(6, 3))

time_in_hours = (df2.index - df2.index[0]) / pd.Timedelta(value=1, unit="h")
sns.lineplot(x=time_in_hours[10:], y="CH 1", data=df2[10:], ax=ax)

ax.set_yscale("log")
ax.set_xlabel("Time")
ax.set_ylabel("Pressure (mbar)")
ax.set_xlabel("Time (in hours)")

fig.tight_layout()
magnet_handler.save_figures(fig, "base_pressure", only_pdf=True)

# Calibration

Data taken from CryoVac email

In [ ]:
calibration = [
    [1, 0.04],
    [5, 0.24],
    [7, 0.34],
    [12, 0.59],
    [16, 0.79],
    [17, 0.84],
    [18, 0.89],
    [19, 0.95],
    [20, 1],
]

In [ ]:
dfc = pd.DataFrame(calibration, columns=["Current", "Flux density"])

In [ ]:
analysis = AnalysisLogic()
fit_x, fit_y, result = analysis.perform_fit(x=dfc["Current"], y=dfc["Flux density"], fit_function=FitMethods.linear)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

sns.scatterplot(x="Current", y="Flux density", data=dfc, ax=ax)
sns.lineplot(x=fit_x, y=fit_y, ax=ax, color="C1", label=f"Slope = {round(result.best_values['slope'], 2)} T/A")


ax.set_xlabel("Current (A)")
ax.set_ylabel("Flux density (T)")

fig.tight_layout()
magnet_handler.save_figures(fig, "magnet_calibration", only_pdf=True)

# Quenching

Data taken from CrypVac email

In [ ]:
quenches_run_1 = [
    [1, 9],
    [2, 10],
    [3, 12.5],
    [4, 13],
    [5, 14],
    [6, 14],
    [7, 15.5],
    [8, 13.5],
    [9, 14],
    [10, 16],
    [11, 16],
    [12, 17],
    [13, 17],
    [14, 18],
    [15, 16.5],
    [16, 17],
    [17, 18.5],
    [18, 18.5],
    [19, 19],
    [20, 18.5],
    [21, 19],
    [22, 18],
    [23, 19],
    [24, 20.5],
    [25, 20.5],
    [26, 20.25],
]

quenches_run_2 = [
    [1, 12],
    [2, 18.5],
    [3, 18.5],
]

In [ ]:
dfq1 = pd.DataFrame(quenches_run_1, columns=["Quench number", "Current"])
dfq2 = pd.DataFrame(quenches_run_2, columns=["Quench number", "Current"])

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

sns.scatterplot(x="Quench number", y="Current", data=dfq1, ax=ax, label="Run 1")
sns.scatterplot(x="Quench number", y="Current", data=dfq2, ax=ax, label="Run 2")

ax.set_xlabel("Quench number")
ax.set_ylabel("Current (A)")

ax.axhline(20, color="C2", linestyle="dashed")

fig.tight_layout()
magnet_handler.save_figures(fig, "magnet_quenches", only_pdf=True)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def draw_ellipsoid(a, b, c):
    coefs = (a, b, c)  # Coefficients in a0/c x**2 + a1/c y**2 + a2/c z**2 = 1 
    # Radii corresponding to the coefficients:
    rx, ry, rz = coefs

    # Set of all spherical angles:
    u = np.linspace(0, 2 * np.pi, 20)
    v = np.linspace(0, np.pi, 20)

    # Cartesian coordinates that correspond to the spherical angles:
    # (this is the equation of an ellipsoid):
    x = rx * np.outer(np.cos(u), np.sin(v))
    y = ry * np.outer(np.sin(u), np.sin(v))
    z = rz * np.outer(np.ones_like(u), np.cos(v))
    
    return x, y, z

In [ ]:
x, y, z = draw_ellipsoid(10, 10, 20)
x2, y2, z2 = draw_ellipsoid(9, 9, 5)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')

ax.contour3D(x, y, z, 30, cmap="viridis")
ax.contour3D(x2, y2, z2)

ax.set_zlabel("$I_z$ (A)")

ax.set_xlim([-20, 20])
ax.set_xlabel("$I_x$ (A)")

ax.set_ylim([-20, 20])
ax.set_ylabel("$I_y$ (A)")

fig.tight_layout()
#magnet_handler.save_figures(fig, "magnet_wireframe")